
# 🌍 Manual TOA Reflectance from Landsat 8

## 🎯 Learning Objectives:
- Load Landsat 8 Level-1 data from Earth Engine
- Understand the process of converting DN values to radiance
- Compute Top-of-Atmosphere (TOA) reflectance from radiance
- Visualize results on an interactive map using `geemap`

---

## 📝 Your Tasks:
1. Load a Landsat 8 Level-1 image over Kraków in July 2021.
2. From metadata, extract the scaling coefficients for each band (RADIANCE_MULT, RADIANCE_ADD).
3. Convert DN to Radiance using the linear formula.
4. Use the ESUN values and solar elevation to convert Radiance to TOA Reflectance.
5. Visualize both the original DN RGB and your TOA Reflectance RGB image.
6. (Optional) Try using a different location or time range.



## 🗺️ Step 1: Define Area of Interest (AOI)

Use Kraków (Poland) as your starting point. You can define this using a point geometry.


In [1]:
# TODO: Define AOI
import ee
ee.Authenticate()
ee.Initialize(project='ee-kamilmolo13')

aoi = ee.Geometry.Point([19.973, 50.094])


## 📦 Step 2: Load Landsat 8 Level-1 Image (Collection 2)

Load the `LANDSAT/LC08/C02/T1_L1` collection, filter for July 2021, and select the image with the least cloud cover.


In [2]:
image = ee.ImageCollection('LANDSAT/LC08/C02/T1') \
    .filterBounds(aoi) \
    .filterDate('2021-07-01', '2022-07-30') \
    .sort('CLOUD_COVER') \
    .first()


## 🔢 Step 3: Convert DN to Radiance

Use the formula:

```
Radiance = M_L * DN + A_L
```

Where:
- `M_L` is `RADIANCE_MULT_BAND_X` from metadata
- `A_L` is `RADIANCE_ADD_BAND_X`
- DN is the digital number of each band

💡 Tip: You can use `image.get('RADIANCE_MULT_BAND_4')` to retrieve the multiplier for band 4.


In [11]:
def DN2Radiance(image: ee.Image, bands:list) -> dict:
    radiance_dict = {}

    for band in bands:
        M_L = image.get(f'RADIANCE_MULT_BAND_{band[1]}').getInfo()
        A_L = image.get(f'RADIANCE_ADD_BAND_{band[1]}').getInfo()

        dn_value = image.select(band)
        
        radiance = dn_value.multiply(M_L).add(A_L)
        radiance_dict[f'{band}'] = radiance

    return radiance_dict
    
bands = ['B2', 'B3', 'B4', 'B5']
radiance_dict = DN2Radiance(image, bands)


## 📊 Step 4: Convert Radiance to TOA Reflectance

Use the following equation from the USGS Landsat Handbook:

```
TOA Reflectance = (π * L_λ * d²) / (ESUN * cos(θ_s))
```

Where:
- `L_λ` = radiance
- `d` = Earth–Sun distance in AU (assume 1.0 for simplification)
- `ESUN` = solar spectral irradiance (use constants from handbook)
- `θ_s` = solar zenith angle = 90° - sun elevation (from image metadata)

Use these ESUN values:
- B2: 2067  
- B3: 1893  
- B4: 1603  
- B5: 972


In [12]:
import math
def Radiance2Reflectance(theta_s: float, radiance_ESUN: dict) -> dict:
    d = 1.0
    reflectance_dict = {}

    for band in radiance_ESUN.keys():
        L_lambda = radiance_ESUN[band][0]
        ESUN = radiance_ESUN[band][1]

        TOA_reflectance = L_lambda.multiply(math.pi*d*d).divide(ESUN * math.cos(math.radians(theta_s)))

        reflectance_dict[f'{band}'] = TOA_reflectance

    return reflectance_dict


radiance_ESUN = \
{
    'B2': [radiance_dict['B2'], 2067.0],
    'B3': [radiance_dict['B3'], 1893.0],
    'B4': [radiance_dict['B4'], 1603.0],
    'B5': [radiance_dict['B5'], 972.0]
}
 
sun_elevation = image.get('SUN_ELEVATION').getInfo()
theta_s = 90.0 - sun_elevation

reflectance_dict = Radiance2Reflectance(theta_s, radiance_ESUN)


## 🖼️ Step 5: Visualize the Results

Use `geemap` to create an interactive map. Display:
- Original RGB from DN bands
- Your manually calculated TOA Reflectance RGB

💡 Hint: Use `.clamp(0, 0.4)` on reflectance bands for better visualization.


In [14]:
import geemap
reflectance_RGB = ee.Image.cat(
    reflectance_dict['B2'].rename('TOA_B2'),
    reflectance_dict['B3'].rename('TOA_B3'),
    reflectance_dict['B4'].rename('TOA_B4'),
    reflectance_dict['B5'].rename('TOA_B5')
    )

Map = geemap.Map()
Map.centerObject(aoi, 12)

Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 10000}, 'RGB') 
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 10000}, 'RGB')
Map.addLayer(reflectance_dict['B4'].clamp(0,0.4),{},'Reflectance B4')
Map.addLayer(reflectance_dict['B3'].clamp(0,0.4), {}, 'Reflectance B3')
Map.addLayer(reflectance_dict['B2'].clamp(0,0.4), {}, 'Reflectance B2')
Map.addLayer(reflectance_RGB.clamp(0,0.4), {'bands': ['TOA_B4', 'TOA_B3', 'TOA_B2'], 'min': 0, 'max': 0.4}, 'Reflectance RGB')

Map

Map(center=[50.094, 19.973], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchData…


## 🧪 Optional Extensions

- Try the same workflow on a different city or biome (e.g., desert, rainforest).
- Add NIR band to your analysis (B5).
- Compute NDVI from TOA or compare with Surface Reflectance data.
- Export the resulting reflectance image to Google Drive.
